In [1]:
#1 reading data and covert to df
from scipy.io import wavfile
import numpy as np
import pandas as pd
import pyreadr

# big to do: read wave values by each eyemovement from wave file, not whole file in dataframe!!!!!!!!

# def data pre process function, 
#wave_seq1["id"] = wave_seq1["id"].astype('float')
#wave_seq1["values"] = wave_seq1["values"].astype('float')
#return a cleaned wave dataframe

#get our dataset: wave_seq(a dataframe combining all eyemovemnt, which is our dataset)
# df col:wave value, id = index of eye movement


In [2]:
wave_seq1 = pd.read_csv("first.csv")

wave_seq1["id"]  = "1"

wave_seq1 = wave_seq1.drop(columns = "Unnamed: 0")
wave_seq1.rename(columns = {'x':'values'}, inplace = True)

wave_seq1["id"] = wave_seq1["id"].astype('float')
wave_seq1["values"] = wave_seq1["values"].astype('float')


wave_seq2 = pd.read_csv("first_1.csv")

wave_seq2["id"]  = "2"

wave_seq2 = wave_seq2.drop(columns = "Unnamed: 0")
wave_seq2.rename(columns = {'x':'values'}, inplace = True)

wave_seq2["id"] = wave_seq2["id"].astype('float')
wave_seq2["values"] = wave_seq2["values"].astype('float')


wave_seq3 = pd.read_csv("r1.csv")

wave_seq3["id"]  = "3"

wave_seq3 = wave_seq3.drop(columns = "Unnamed: 0")
wave_seq3.rename(columns = {'x':'values'}, inplace = True)

wave_seq3["id"] = wave_seq3["id"].astype('float')
wave_seq3["values"] = wave_seq3["values"].astype('float')


wave_seq4 = pd.read_csv("r2.csv")

wave_seq4["id"]  = "4"

wave_seq4 = wave_seq4.drop(columns = "Unnamed: 0")
wave_seq4.rename(columns = {'x':'values'}, inplace = True)

wave_seq4["id"] = wave_seq4["id"].astype('float')
wave_seq4["values"] = wave_seq4["values"].astype('float')

wave_seq = pd.concat([wave_seq1,wave_seq2,wave_seq3,wave_seq4],ignore_index=True)

# downsampling


print(wave_seq)



       values   id
0        21.0  1.0
1        19.0  1.0
2        18.0  1.0
3        17.0  1.0
4        16.0  1.0
...       ...  ...
64401   -48.0  4.0
64402   -49.0  4.0
64403   -51.0  4.0
64404   -51.0  4.0
64405   -53.0  4.0

[64406 rows x 2 columns]


In [170]:
# downsampling

def downsample_dataframe(dataframe, new_sample_rate):
    downsampling_factor = int(dataframe.shape[0] / new_sample_rate)
    downsampled_dataframe = dataframe.iloc[::downsampling_factor, :]
    return downsampled_dataframe


def downsample_process(inputdf, ds_df):
    if inputdf.shape[0] - ds_df.shape[0] != 0:
        return print("downsample complete")
    else:
        return print("downsample not work")
    
downsampled_df = downsample_dataframe(wave_seq, 1000)


In [274]:
#2. feature extraction

#pip install tsfresh
from tsfresh import extract_features 
from tsfresh import extract_relevant_features
from tsfresh.feature_extraction import extract_features, EfficientFCParameters
from tsfresh.feature_extraction import ComprehensiveFCParameters
from tsfresh.feature_extraction import MinimalFCParameters, settings

import tsfresh

ext_feature_df = extract_features(downsampled_df,column_id="id",
                                  default_fc_parameters=ComprehensiveFCParameters(),
                                  n_jobs=0)

ext_feature_clean = ext_feature_df.dropna(axis = 1, how = "any")
print(ext_feature_clean)



Feature Extraction: 100%|█████████████████████████| 4/4 [00:00<00:00,  8.98it/s]

     values__variance_larger_than_standard_deviation  \
1.0                                              1.0   
2.0                                              1.0   
3.0                                              1.0   
4.0                                              1.0   

     values__has_duplicate_max  values__has_duplicate_min  \
1.0                        0.0                        0.0   
2.0                        0.0                        0.0   
3.0                        0.0                        0.0   
4.0                        0.0                        0.0   

     values__has_duplicate  values__sum_values  values__abs_energy  \
1.0                    1.0            -14601.0          94573425.0   
2.0                    1.0              8256.0         155722516.0   
3.0                    1.0             -8953.0         410779937.0   
4.0                    1.0            -12534.0         383789288.0   

     values__mean_abs_change  values__mean_change  \
1.0      

In [275]:
# data normalization,ignore all warning

from sklearn.preprocessing import minmax_scale
cols = ext_feature_clean.select_dtypes(np.number).columns
ext_feature_clean[cols] = minmax_scale(ext_feature_clean[cols])
ext_feature_clean.head()


/var/folders/nn/h99dr9455yg83gkxbsycq7m40000gn/T/ipykernel_50136/3051204813.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ext_feature_clean[cols] = minmax_scale(ext_feature_clean[cols])


,values__variance_larger_than_standard_deviation,values__has_duplicate_max,values__has_duplicate_min,values__has_duplicate,values__sum_values,values__abs_energy,values__mean_abs_change,values__mean_change,values__mean_second_derivative_central,values__median,...,values__fourier_entropy__bins_3,values__fourier_entropy__bins_5,values__fourier_entropy__bins_10,values__fourier_entropy__bins_100,values__permutation_entropy__dimension_3__tau_1,values__permutation_entropy__dimension_4__tau_1,values__permutation_entropy__dimension_5__tau_1,values__permutation_entropy__dimension_6__tau_1,values__permutation_entropy__dimension_7__tau_1,values__mean_n_absolute_max__number_of_maxima_7
1.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.800185,1.000000,1.000000,0.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2.0,0.0,0.0,0.0,0.0,1.000000,0.193383,1.000000,0.473259,0.613680,0.830239,...,0.649919,0.610735,0.762944,0.674596,0.402455,0.871087,0.763492,0.715084,0.576456,0.150188
3.0,0.0,0.0,0.0,0.0,0.247102,1.000000,0.214757,0.513885,0.221325,1.000000,...,0.000000,0.367919,0.305814,0.294449,0.000000,0.772673,0.786358,0.931587,0.918850,1.000000
4.0,0.0,0.0,0.0,0.0,0.090432,0.914642,0.000000,0.000000,0.000000,0.628647,...,0.000000,0.000000,0.000000,0.000000,0.743453,1.000000,1.000000,1.000000,1.000000,0.658207


In [276]:
# 3.0 feature selection
main = ext_feature_clean
Y = ["L","L","R","R"]#label
main["label"] = Y

from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import VarianceThreshold

# Separate the features (X) and the target variable (y)
X = main.drop(columns="label")
y = main["label"]

# Encode the target variable to numeric values using LabelEncoder
le = LabelEncoder()
y_encoded = le.fit_transform(y)

# Remove constant or near-constant features
filter1 = VarianceThreshold(threshold=0.1)
X_filter1 = filter1.fit_transform(X)

# Select the top k features using SelectKBest and f_regression
filter2 = SelectKBest(score_func=f_regression, k=50)
X_filter2 = filter2.fit_transform(X_filter1, y_encoded)

# Get the selected features
filter_inx = filter2.get_support(indices=True)
selected_features = X.columns[filter_inx]

# Print the selected feature names
ind = ['label'] + list(selected_features)
selected_features_df = main[ind]

print(selected_features_df)

    label  values__has_duplicate_max  values__root_mean_square  \
1.0     L                        0.0                  0.000000   
2.0     L                        0.0                  0.260626   
3.0     R                        0.0                  1.000000   
4.0     R                        0.0                  0.798402   

     values__last_location_of_maximum  values__first_location_of_maximum  \
1.0                          1.000000                           1.000000   
2.0                          0.000000                           0.000000   
3.0                          0.919085                           0.924302   
4.0                          0.984648                           0.990402   

     values__percentage_of_reoccurring_values_to_all_values  \
1.0                                           0.000000        
2.0                                           0.176825        
3.0                                           0.926766        
4.0                                 

/var/folders/nn/h99dr9455yg83gkxbsycq7m40000gn/T/ipykernel_50136/2571192670.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  main["label"] = Y
/var/folders/nn/h99dr9455yg83gkxbsycq7m40000gn/T/ipykernel_50136/2571192670.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  main["label"] = Y
/Applications/anaconda/anaconda3/lib/python3.9/site-packages/sklearn/feature_selection/_univariate_selection.py:358: RuntimeWarning: divide by zero encountered in true_divide
  f_statistic = corr_coef_squared / (1 - corr_coef_squar

In [278]:
#final data check 
#clean_df = selected_features_df.replace([np.inf, -np.inf], np.nan).dropna(axis=1,how = "any")
check_for_nan = selected_features_df.isnull().sum().sum()
print(check_for_nan)


0


In [279]:
#3. create classifier-RF, def function, return rf.fit(X, Y)
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
import numpy as np 

# Separate the features (X) and the target variable (y)
X = selected_features_df.drop(columns="label")
y = selected_features_df["label"]



In [280]:
rf = RandomForestClassifier()
rf.fit(X,  y)

RandomForestClassifier()

In [298]:
#4. predict for event, call (def data pre process function)

#input feature extraction
input_features = pd.read_csv("first.csv")
input_features["id"]  = "0"
input_features = input_features.drop(columns = "Unnamed: 0")
input_features.rename(columns = {'x':'values'}, inplace = True)
input_features["id"] = input_features["id"].astype('float')
input_features["values"] = input_features["values"].astype('float')

downsampled_input1 = downsample_dataframe(input_features, 1000)

downsample_process(input_features, downsampled_input1)

input_features_df1 = extract_features(downsampled_input1,column_id="id",
                                  default_fc_parameters=ComprehensiveFCParameters(),
                                  n_jobs=0)


downsample complete


Feature Extraction: 100%|█████████████████████████| 1/1 [00:00<00:00,  1.12it/s]


In [299]:
#input fs
ind =list(selected_features)
input_fs_df1= input_features_df1[ind]


In [300]:

def check_dim(df1, df2):
    if df1.shape[1]-df2.shape[1] == 0:
        return print("dimension matched")
    else:
        return print("check the dim")
    
check_dim(X,input_fs_df1)

dimension matched


In [302]:
#testing prediction
# L test
a= rf.predict(input_fs_df1)#dataset in live
prediction = a[0]
print(prediction)


L


In [304]:
# 5.0 Evaluation
import re
import librosa
import numpy as np
from sklearn import svm
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score


    
folds = 4
repeats = 10
seed = 3888
np.random.seed(seed)

kf = KFold(n_splits=folds, shuffle=True, random_state=seed)
acc50_svm = []
acc50_rf = []
X = np.array(X)
for _ in range(repeats):
    acc_svm = []
    acc_rf = []

    for train_index, test_index in kf.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = Y[train_index], Y[test_index]

        # SVM
        svm_res = svm.NuSVC(kernel='linear')
        svm_res.fit(X_train, y_train)
        y_pred_svm = svm_res.predict(X_test)
        acc_svm = accuracy_score(y_test, y_pred_svm)
        acc_svm.append(acc_svm)
        
        # RF
        rf = RandomForestClassifier()
        rf.fit(X, Y)
        y_pred_rf = rf.predict(X_test)
        acc_rf = accuracy_score(y_test, y_pred_rf)
        acc_rf.append(acc_rf)
        
        # 

    acc50_svm.append(np.mean(acc_svm))
    acc50_rf.append(np.mean(acc_rf))

avg1 = np.mean(acc50_svm)
avg2 = np.mean(acc50_rf)
print(avg1)
print(avg2)


TypeError: only integer scalar arrays can be converted to a scalar index